# Objective
Predict resale prices of BMW cars. This could for instance be used by someone who wants to sell their car, to get an idea about how much it is worth, similar to how Kelley Blue Book works.

# Thinking about the problem
From the readme of the dataset available here https://github.com/datacamp/careerhub-data/tree/master/BMW%20Used%20Car%20Sales, one can see that the dataset contains information about price, transmission, mileage, fuel type, road tax, miles per gallon (mpg), and engine size. Upon insepction of the dataset (see below), it turned out to additionally contain the car model and year (I'm assuming this means production year). First I want to describe my initial expectations for the relationships between these quantites, and formulate different levels of complexity for including the data.

The five quantites model, year, transmission, fuel type, and engine size collectively describe the car configuration at the time of initial purchase. The quantity milage describes how much the car has been used, and therefore worn since that point. The quantities miles per gallon and road tax should be given based on the new car configuration quantities.

I suspect that the price will strongly depend on the mileage and age of the car, and a first simple model could therefore just consider these two variables.

An improvement on this would be to include the new car configuration variables. From these in addition to price, mpg and road tax could be inferred.

Finally the last two variables, mpg and road tax, can be included. These could affect the resale price of the car, since they would probably influence how much a buyer is willing to pay, but I suspect this connection will be less strong than the connection between the other variables and price.

Before any of this though, first I want to take a closer at the data.

In [ ]:
from diagrams import Diagram
from diagrams.aws.compute import EC2
from diagrams.aws.database import RDS
from diagrams.aws.network import ELB
from diagrams.custom import Custom

with Diagram("Web Service", show=False) as diag:
    Custom("mpg", None) >> Custom("engine size", None)

diag

In [ ]:
from graphviz import Digraph, Graph
dot = Digraph(comment='First data model')
dot.node('m', 'mileage')
dot.node('p', 'price')
dot.edges(['mp'])
dot.graph_attr['rankdir'] = 'LR'
dot

In [ ]:
from itertools import combinations
dot = Graph(comment='Data model 2', engine='fdp'
           )

def draw_new_and_usage_clusters(dot):
    new_car_config = ['model', 'transmission', 'fuelType', 'engineSize', 'year']
    with dot.subgraph(name='clusterA') as c:
        c.attr(style='filled', color='lightgrey', shape='egg')
        for node in new_car_config:
            c.node(node)
        c.node_attr.update(style='filled', color='white')
        #for pair in combinations('mtfey', r=2):
        #    c.edge(*pair)
        #c.edges([('a0', 'a1'), ('a1', 'a2'), ('a2', 'a3')])
        c.attr(label='New car configuration')

    with dot.subgraph(name='clusterB') as c:
        c.attr(style='filled', color='lightgrey')
        c.node_attr.update(style='filled', color='white')
        c.node('mileage')
        c.attr(label='Usage')

        
draw_new_and_usage_clusters(dot)

features = ['price', 'tax', 'mpg']
short = 'pag'

with dot.subgraph(name='clusterC') as c:
    c.attr(style='filled', color='lightgrey')
    c.node_attr.update(style='filled', color='white')
    for node in features:
        c.node(node)
    c.attr(label='Predictables')

    
#for s, feature in zip(short, features):
#    dot.node(s, feature)

dot.edge('clusterA', 'mpg', dir='forward')
dot.edge('clusterA', 'tax', dir='forward', splines='ortho')
dot.edge('clusterA', 'price', dir='forward')
dot.edge('clusterB', 'price', dir='forward',
                            splines='curved')
#dot.edge('clusterA', 'clusterC', dir='forward')
#dot.edge('clusterB', 'clusterC', dir='forward')
dot.graph_attr['rankdir'] = 'LR'
#dot.unflatten()
dot


In [ ]:
from itertools import combinations
dot = Graph(comment='Data model 3', engine='fdp',
           )

draw_new_and_usage_clusters(dot)

features = ['price', 'tax', 'mpg']
short = 'pag'

with dot.subgraph(name='clusterC') as c:
    c.attr(style='filled', color='lightgrey')
    c.node_attr.update(style='filled', color='white')
    for node in ['tax', 'mpg']:
        c.node(node)
    c.attr(label='Others')


    
dot.edge('clusterA', 'price', dir='forward')
dot.edge('clusterB', 'price', dir='forward')
dot.edge('clusterC', 'price', dir='forward', color='grey',
)
#dot.edge('clusterA', 'clusterC', dir='forward')
#dot.edge('clusterB', 'clusterC', dir='forward')
#dot.graph_attr['rankdir'] = 'LR'

dot



# Loading and inspecting data
First I load and inspect the data. I downloaded the data from [here](https://raw.githubusercontent.com/datacamp/careerhub-data/master/BMW%20Used%20Car%20Sales/bmw.csv) and saved it in the `datasets/bmw.csv` file.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
bmw = pd.read_csv('datasets/bmw.csv')
bmw.head()

In [ ]:
bmw.info()

In [ ]:
bmw.model.unique()

In [ ]:
bmw.transmission.unique()

In [ ]:
bmw.fuelType.unique()

In [ ]:
bmw.describe()

In [ ]:
for col in bmw:
    print(col, len(bmw[col].unique()))

# Data cleaning
Let us take a closer look at the categorical axes. First we print the number of values in each category

In [ ]:
categorical_columns= ['model', 'fuelType', 'transmission']
def print_categorical_counts(df, columns):
    for col in columns:
        display(df.groupby(col)[col].count())

print_categorical_counts(bmw, categorical_columns)

There are  have a number of categories with very few records. For instance, the `fuelType` `Electric` has only three. With such a small amount of observations for this category, and no obvious relationship with other entries in this category as one naturally has for numeric columns, I wouldn't expect it to be possible to make reliable predictions for the selling price for this category. I therefore choose to drop any category with less than 10 records. 

In [ ]:
def drop_almost_empty_categories(df, col, nmin=10):
    df = df.copy() # To avoid modyfiyng the input dataframe
    category_count = df.groupby(col)[col].count()
    for category_name, count  in category_count.iteritems():
        if count < nmin:
            df = df[df[col] != category_name]
    return df

bmw_dropped = bmw.copy()
for col in categorical_columns:
    bmw_dropped = drop_almost_empty_categories(bmw_dropped, col)
#print_categorical_counts(bmw_dropped, categorical_columns)

In [ ]:
new_car_config_cols = ['model', 'transmission', 'fuelType', 'engineSize']
new_car_cols = new_car_config_cols + ['year']

In [ ]:
bmw_dropped[bmw_dropped.engineSize==0].head()

In [ ]:
with pd.option_context('display.max_rows', None):
    new_car_grouped = bmw.groupby(new_car_cols)[['tax', 'mpg', 'price']]
    display(new_car_grouped.nunique())
    #display(bmw.groupby(new_car_config_cols)['tax'].nunique())

In [ ]:
choices = (bmw.model==' 1 Series') & (bmw.transmission=='Automatic') & (bmw.fuelType=='Diesel') &(bmw.engineSize==2.0) & (bmw.year==2016)
bmw[choices][["mileage", 'tax', 'mpg', 'price']].sort_values('mileage')

# Data exploration
<a id = "data-exploration"></a>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.pairplot(bmw_dropped.sort_values("engineSize"), #hue='transmission', 
             x_vars=new_car_cols,
             y_vars=new_car_cols,)

In [ ]:
sns.pairplot(bmw_dropped, #hue='transmission', 
             x_vars=['price', 'year', 'mileage',  'tax', 'mpg', 'engineSize'],
             y_vars=['price'],)

In [ ]:
bmw_copy = bmw_dropped.copy()
#bmw_copy['ln mileage'] = np.log(bmw_copy['mileage'])
#bmw_copy['-mileage'] = -bmw_copy['mileage']
#bmw_copy['10^mileage'] = np.exp(bmw_copy['mileage'])
bmw_copy['ln price'] = np.log(bmw_copy['price'])
bmw_copy = bmw_copy.drop('price', axis='columns')
#bmw_copy = bmw_copy.drop('mileage', axis='columns')

sns.pairplot(bmw_copy, #hue='transmission', 
             x_vars=['ln price', 'year', 'mileage',  'tax', 'mpg', 'engineSize'],
             y_vars=['ln price']) #, hue='transmission')

In [ ]:
bmw_copy.corr()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=[16, 4])
sns.kdeplot(x='price', data=bmw_copy, ax=axes[0])
sns.kdeplot(x='ln price', data=bmw_copy, ax=axes[1])

In [ ]:
plt.figure(figsize=(16, 6))
sns.violinplot(y='price', x='model', data=bmw_dropped, aspect=2)

## A bit more data cleaning

In [ ]:
bmw[bmw["mpg"] > 400]

In [ ]:
bmw[bmw["tax"] <= 0]

In [ ]:
bmw[bmw["fuelType"]=='Hybrid'].groupby("mpg")["mpg"].count()

From the plots we can see that `mpg` has a group of values near 400, far from the nearest values who are less than 200. Let's see how many different values  are present there

In [ ]:
bmw_dropped[bmw_dropped["mpg"]>400]["mpg"].unique()

All the values of `mpg` in the group near 400 have the same value. This looks very suspicious. I suspect this is data is wrong, and since it could seriously skew a model since it has such high values, I should eliminate these values (either impute with e.g. average, or drop the records all together).

Let's also check the remaining two continous variables

In [ ]:
#display(sorted(bmw_dropped["engineSize"].unique()))
display(bmw_dropped.groupby("engineSize")["engineSize"].count())
bmw_dropped.groupby("tax")["tax"].count()

They both contain zeros, which seems weird for both tax and engine size. The skewing effect is probably less then for the `mpg` outliers, since zero is closer to other values of tax and engine size, but I should still either impute or drop these records.

# Linear regression model 1
For the first model, I only want to consider the dependency of price on build year and mileage. From the plots in the [data exploration](#data-exploration) section we see that the logarithm of the price appears to depend linearly on year and mileage.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

def split_dependent(df, dependent="ln price"):
    other_cols = [col for col in df.columns if col != dependent]
    return df[other_cols], df[dependent]

def fit_and_test_linear_model(bmw_selected):

    linreg = LinearRegression()
    features, linreg = fit_and_test_model(bmw_selected, linreg)
    #print(features)
    coeffs = pd.DataFrame({"observable": features, "coef": linreg.coef_, "exp coef": np.exp(linreg.coef_)})
    display(coeffs.set_index("observable"))

    # Plot outputs
    #print(X_test)
    #plt.scatter(X_test["mileage"], price_test,  color='black')
    #plt.plot(X_test["mileage"], price_predict, color='blue', linewidth=3)
    #sns.scatterplot(x=X_test["mileage"], y=price_test, alpha=0.5)
    #sns.lineplot(x=X_test["mileage"], y=price_predict, alpha=0.5, color='red')
    #sns.scatterplot(x=price_test, y=price_predict, alpha=0.5)

    #plt.show()
    return linreg

def fit_and_test_model(bmw_selected, model, dependent='ln price'):

    bmw_train, bmw_test = train_test_split(bmw_selected, test_size=0.1, random_state=42)

    model.fit(*split_dependent(bmw_train, dependent=dependent))

    X_test, y_test = split_dependent(bmw_test.sort_values("mileage"), dependent=dependent)

    y_predict = model.predict(X_test)

    # The mean squared error
    print('Mean squared error: %.2f'
          % mean_squared_error(y_predict, y_test))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
          % r2_score(y_predict, y_test))

    sns.scatterplot(x=y_test, y=y_predict, alpha=0.5)
    plt.show()
    return X_test.columns, model


In [ ]:
bmw_selected = bmw_copy[["ln price", "year", "mileage"]]
fit_and_test_linear_model(bmw_selected)

In [ ]:
bmw_selected = bmw_copy[["ln price", "mileage"] + new_car_cols]
bmw_selected = pd.get_dummies(bmw_selected, drop_first=True)
fit_and_test_linear_model(bmw_selected)

In [ ]:
bmw_selected = pd.get_dummies(bmw_copy, drop_first=True)
fit_and_test_linear_model(bmw_selected)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()
bmw_selected = bmw_copy[["ln price", "mileage"] + new_car_cols]
bmw_selected = pd.get_dummies(bmw_selected, drop_first=True)
fit_and_test_model(bmw_selected, gbr)